# Steuerungsentwurf

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from IPython.display import display, Image, Latex
from scipy.integrate import solve_ivp

In [ ]:
imag = Image("../images/feedforward.png", width=600)
display(imag)

Ziel ist der Entwurf einer Steuerung auf Basis des linearisierten Modells des Zweitanksystems
\begin{align*}
    T_1 T_2 \ddot{\tilde{z}}_2(t) + (T_1 + T_2)\dot{\tilde{z}}_2(t) + \tilde{z}_2(t) & = \tilde{u}_{\mathrm{A}}(t)
\end{align*}
um die Füllstandshöhe des zweiten Tanks
\begin{align*}
    z_{2,\mathrm{r}}(t) & = \begin{cases}
         0\, \mathrm{m} & t \leq 0 \\
         \varphi(t) & 0 < t < \Delta t \\
         0,15\, \mathrm{m} & t \geq \Delta t \\
    \end{cases}
\end{align*}
in einer vorgegebenen Zeit $\Delta t = 50\, \mathrm{s}$ von einer Höhe auf eine andere Höhe zu überführen.
Dazu ist eine $n$-mal stetig differenzierbaren Trajektorie $t \mapsto \varphi(t)$ zu bestimmen aus der das Referenzeingangssignal $t \mapsto u_{\mathrm{A,r}}(t)$ berechnet wird.

### 1. Eintragen der notwendigen Systemparameter

In [ ]:
buA = 9
# Physikalisch
KPhy = 0.147763421835044
T1Phy = 34.567259359529
T2Phy = 68.2115206317666
bz2Phy = 0.137092448385557
# Messung
KMessung = 0.05335540886113588
T1Messung = 11.796537856412515
T2Messung = 80.74097989401139
bz2Messung = 0.13238994944989588

### 2. Bestimmung der Trajektorie

In [ ]:
# Ordnung des Systems
n = 2
# Grad der Trajektorie
q = 2 * n + 1
# Definition der benötigten Symbole
t, dt, yd0, yd1 = sp.symbols('t \\Delta y_{d0} y_{d1}')
bc = sp.symbols('c0:%d' % (q + 1))
phi = []
# Definition der numerischen Werte
dtNum = 150
yd0Num = 0. - bz2Messung
yd1Num = 0.15 - bz2Messung

phi.append(sum(bc_*t**i for i, bc_ in enumerate(bc)))
display(Latex("$\\varphi(t) = {}$".format(sp.latex(phi[0]))))

for i in range(n):
    phi.append(phi[i].diff(t))
    display(Latex("$\\varphi^{{({})}}(t) = {}$".format(i + 1, sp.latex(phi[i+1]))))

In [ ]:
sys = []
for i in range(n + 1):
    sys.append(phi[i].subs({t: 0}))
    sys.append(phi[i].subs({t: dt}))
    if i == 0:
        sys[-2] -= yd0
        sys[-1] -= yd1
        
res = sp.solve(sys, bc)
bcNum = [list(res.items())[i][1].subs({yd0: yd0Num, yd1: yd1Num, dt: dtNum}) for i in range(q + 1)]
for i in range(q + 1):
    display(Latex("${} = {} = {}$".format(sp.latex(list(res.items())[i][0]), sp.latex(list(res.items())[i][1]), bcNum[i])))

Implementierung der eigentlichen Trajektorie mit Referenzeingang

In [ ]:
a0 = 1 / (T1Messung * T2Messung)
a1 = (T1Messung + T2Messung) / (T1Messung * T2Messung)
b = KMessung / (T1Messung * T2Messung)
(c0, c1, c2, c3, c4, c5) = bcNum

yr = lambda t: c0 + c1 * t + c2 * t ** 2 + c3 * t ** 3 + c4 * t ** 4 + c5 * t ** 5 if 0 <= t <= dtNum else yd0Num if t < 0 else yd1Num 
dyr = lambda t: c1 + 2 * c2 * t + 3 * c3 * t ** 2 + 4 * c4 * t ** 3 + 5 * c5 * t ** 4 if 0 < t < dtNum else 0
ddyr = lambda t: 2 * c2 + 6 * c3 * t + 12 * c4 * t ** 2 + 20 * c5 * t ** 3 if 0 < t < dtNum else 0
uAr = lambda t: 1 / b * (ddyr(t) + a1 * dyr(t) + a0 * yr(t)) + buA

### 3. Vergleich der Steuerung an den Modellen

__Simulation__

Definition nichtlineares Modell

In [ ]:
def nonLinSys(t, z, uA):
    z1 = z[0]
    z2 = z[1]
        
    hV1 = 0.055
    hV2 = 0.055
    
    dz = np.zeros(2)
    dz[0] = 0.00216625315586689 * (uA(t) - 6.4) - 0.0127646468529449 * np.sqrt(2) * np.sqrt(hV1 + z1)
    dz[1] = 0.0127646468529449 * np.sqrt(2) * np.sqrt(hV1 + z1) - 0.00908683019582126 * np.sqrt(2) * np.sqrt(hV2 + z2)

    return dz

Definition linearisiertes Modell

In [ ]:
def linSys(t, x, uA, T1Sys, T2Sys, KSys, buA):
    A = np.array([[0, 1],
                  [-1 / (T1Sys * T2Sys), -(T1Sys + T2Sys) / (T1Sys * T2Sys)]])
    B = np.array([[0],
                  [KSys / (T1Sys * T2Sys)]])
    return A.dot(x) + B.dot(np.array([uA(t) - buA]))

In [ ]:
timeDom = np.linspace(0, 200, 2000)
z0 = [0, 0]

resPhyLin = solve_ivp(linSys, [timeDom[0], timeDom[-1]], z0 - np.array([bz2Phy, 0]), t_eval=timeDom, args=(uAr, T1Phy, T2Phy, KPhy, buA))
resPhyNichtLin = solve_ivp(nonLinSys, [timeDom[0], timeDom[-1]], z0, t_eval=timeDom, args=(uAr, ))
resMessung = solve_ivp(linSys, [timeDom[0], timeDom[-1]], z0 - np.array([bz2Messung, 0]), t_eval=timeDom, args=(uAr, T1Messung, T2Messung, KMessung, buA))

In [ ]:
plt.close()

fig1, axes10 = plt.subplots(1, 1, figsize=(12,7))

axes10.plot(timeDom, resPhyNichtLin.y[1], label='physikalisch nichtlinear')
axes10.plot(timeDom, resPhyLin.y[0] + bz2Phy, label='physikalisch linearisiert')
axes10.plot(timeDom, resMessung.y[0] + bz2Messung, label=r'identifiziert')

axes11 = axes10.twinx() 

axes11.plot(timeDom, [uAr(t) for t in timeDom], color='C4')
axes11.tick_params(axis='y', labelcolor='C4')

axes10.set_ylabel(r'$z_2$ / $m$]')
axes11.set_ylabel(r'$u_{\mathrm{A}}$ / $V$', color='C4')
axes10.set_xlabel(r'Zeit / $s$')

handlesAx1, labelsAx1 = axes10.get_legend_handles_labels()
axes10.legend([handle for i, handle in enumerate(handlesAx1)],
              [label for i, label in enumerate(labelsAx1)],
              bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
              ncol=9, mode="expand", borderaxespad=0., framealpha=0.5)

axes10.grid()
plt.show()